# identify the users who run the hardest jobs
We want to know who on average runs heavy jobs. We need to take into account that some users do not run many jobs

In [1]:
# import  epmt query 
print('importing epmt_query')
import epmt_query as eq
# import matplot for better plotting functions
import sys
sys.path.insert(0,'/home/Avery.Kiihne/pip_experiment')
import matplotlib.pyplot as plt
import numpy as np
# import pandas. optional but helpful 'display.max_columns' arg shows all DataFrame columns when printing
print('importing pandas')
import pandas
pandas.set_option('display.max_columns', None)

importing epmt_query
importing pandas


In [2]:
import time
start = time.time()
limiter = eq.get_jobs(limit = 200,after=-12, before = -5, fmt = 'orm')
job_num = limiter.count()   #number of jobs grabbed
jobs_all = []
for aa in range(10):
    jobs_temp = eq.get_jobs(limit=int(job_num/10), after = -12, before = -5, offset = int(job_num*aa/10), 
                           fmt='dict')
    jobs_all.extend(jobs_temp)
    print('at ',(aa+1)*10,'% completion. time elapsed:',(time.time()-start)/60)
print('jobs in this time period: ', len(jobs_all))

at  10 % completion. time elapsed: 0.0009807149569193523
at  20 % completion. time elapsed: 0.0013513525327046713
at  30 % completion. time elapsed: 0.0017491380373636882
at  40 % completion. time elapsed: 0.002156047026316325
at  50 % completion. time elapsed: 0.0025076746940612795
at  60 % completion. time elapsed: 0.002843459447224935
at  70 % completion. time elapsed: 0.003200423717498779
at  80 % completion. time elapsed: 0.003626588980356852
at  90 % completion. time elapsed: 0.004080911477406819
at  100 % completion. time elapsed: 0.004501028855641683
jobs in this time period:  200


In [3]:
users = []
for job in jobs_all:
    users.append(job['user'])

In [4]:
users = set(users)
totals = {}
scores = []
final_score = {}
for person in users:
    totals[person] = [0,0,0]   #rssmax, cpu_time, number of jobs
    final_score[person] = 0
#sort data into each person. we care about rssmax, cpu_time, and total number of jobs
for job in jobs_all:
    if job.get('rssmax'):
        totals[job['user']][0] += job['rssmax']
        totals[job['user']][1] += job['cpu_time']
        totals[job['user']][2] += 1
for person in users:
    totals[person][0] = totals[person][0]/totals[person][2]
    totals[person][1] = totals[person][1]/totals[person][2]
    scores.append([person,totals[person][0],totals[person][1], totals[person][2]])
#sort rankings
num_jobs_ranking = sorted(scores, key=lambda score: score[3], reverse=True)
num_jobs_ranking = num_jobs_ranking[0:int(.7*len(num_jobs_ranking))]   #elimate the bottom 30% based on jobs run
rssmax_ranking = sorted(num_jobs_ranking, key=lambda score: score[1], reverse=True)
cpu_time_ranking = sorted(num_jobs_ranking, key=lambda score: score[2], reverse=True)

In [5]:
#tally final score based on ranking of rssmax and cpu_time averages totalled
final_score = {}
for aa in range(len(num_jobs_ranking)):   #make keys fro score dictionary
    final_score[num_jobs_ranking[aa][0]] = 0
for count in range(len(num_jobs_ranking)):   #combine score
    final_score[rssmax_ranking[count][0]] += count
    final_score[cpu_time_ranking[count][0]] += count
#wrap into list because dictionaries do not sort easily
final_rank = []
for bb in range(len(num_jobs_ranking)):   #make keys fro score dictionary
    final_rank.append([final_score[num_jobs_ranking[bb][0]], num_jobs_ranking[bb][0]])
final_rank = sorted(final_rank)

In [17]:
print('the users with the heaviest average usage are ',final_rank[0][1] ,' and ' , final_rank[1][1] + ' and ' , final_rank[2][1])
print('From ',jobs_all[0]['updated_at'] , ' to ',jobs_all[-1]['updated_at'])

the users with the heaviest average usage are  David.Paynter  and  Raphael.Dussin and  Eric.Stofferahn
From  2024-03-03 15:47:40.576806  to  2024-03-20 16:32:31.864116


In [6]:
final_rank

[[1, 'Eric.Stofferahn'],
 [1, 'oar.gfdl.am5'],
 [4, 'John.Krasting'],
 [6, 'Liwei.Jia'],
 [8, 'Oar.Gfdl.Nmme'],
 [10, 'Cole.Harvey']]

In [7]:
rssmax_ranking

[['oar.gfdl.am5', 16685321714.75, 1537367085.625, 16],
 ['Eric.Stofferahn', 266454612.0, 1978626305.0, 1],
 ['John.Krasting', 44385380.0, 648516717.0, 1],
 ['Liwei.Jia', 41585998.24, 91848618.06, 50],
 ['Oar.Gfdl.Nmme', 24977957.37142857, 43610842.01428571, 70],
 ['Cole.Harvey', 4005768.474576271, 19843880.101694915, 59]]

In [8]:
cpu_time_ranking

[['Eric.Stofferahn', 266454612.0, 1978626305.0, 1],
 ['oar.gfdl.am5', 16685321714.75, 1537367085.625, 16],
 ['John.Krasting', 44385380.0, 648516717.0, 1],
 ['Liwei.Jia', 41585998.24, 91848618.06, 50],
 ['Oar.Gfdl.Nmme', 24977957.37142857, 43610842.01428571, 70],
 ['Cole.Harvey', 4005768.474576271, 19843880.101694915, 59]]

In [13]:
num_jobs_ranking[0]

['Oar.Gfdl.Nmme', 24977957.37142857, 43610842.01428571, 70]

In [11]:
labels = []
sizes = []
for x, y in cpu_time_ranking.items():
    labels.append(x)
    sizes.append(y)

AttributeError: 'list' object has no attribute 'items'